In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import time
from skimage import io
from skimage import transform

import keras
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, BatchNormalization, Dropout, MaxPool2D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Lambda

In [ ]:
BUFFER_SIZE = 8000
BATCH_SIZE = 10

filenames = ['cars_train_resized/car (' + str(j + 1) + ').jpg' for j in range(BUFFER_SIZE)]
images_gen = io.imread_collection(filenames)

In [ ]:
images_gen = np.asarray(images_gen)
images_gen = images_gen / 255
print(images_gen.shape)

In [ ]:
hidden_dim = 25
batch_size = 10
BATCH_SIZE = 10
train_dataset = tf.data.Dataset.from_tensor_slices(images_gen).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
def dropout_and_batch(x):
    return Dropout(0.3)(BatchNormalization()(x))

def add(x):
    return tf.concat(x, -1)

print(batch_size)

In [ ]:
filenames = ['cars_train_resized/car (' + str(j + 1) + ').jpg' for j in range(1157)]
images = io.imread_collection(filenames)
filenames = ['waste_resized/waste (' + str(j + 1) + ').jpg' for j in range(1199)]
images_1 = io.imread_collection(filenames)
answer = [1 for j in range (1157)]
answer_1 = [0 for j in range (1199)]
answer = np.concatenate((np.asarray(answer), np.asarray(answer_1)))
images = np.concatenate((np.asarray(images), np.asarray(images_1)))
images.shape

In [ ]:
answer = answer[:2355]
idx = np.random.permutation(len(answer))
images,answer = images[idx], answer[idx]
train = images[:2100] / 255
test = images[2100:] / 255
train_answer = answer[:2100] 
test_answer = answer[2100:] 

In [ ]:
discriminator = tf.keras.Sequential()
discriminator.add(Conv2D(64, (9, 9), strides=(1, 1), padding='same', input_shape=[100, 160, 3]))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.2))

discriminator.add(MaxPool2D(pool_size=(2, 2)))

discriminator.add(Conv2D(128, (7, 7), strides=(1, 1), padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.25))

discriminator.add(MaxPool2D(pool_size=(2, 2)))

discriminator.add(Conv2D(128, (5, 5), strides=(1, 1), padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.25))

discriminator.add(Conv2D(64, (5, 5), strides=(1, 1), padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.25))

discriminator.add(MaxPool2D(pool_size=(2, 2)))

discriminator.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.25))

discriminator.add(MaxPool2D(pool_size=(2, 2)))

discriminator.add(Flatten())
discriminator.add(Dense(1024))
discriminator.add(Dense(1, activity_regularizer=tf.keras.regularizers.l2(0.0001)))

discriminator.compile(
    optimizer=tf.keras.optimizers.SGD(),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
input_img = Input((100, 160, 3))
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(input_img)
x = LeakyReLU()(x)
x = dropout_and_batch(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Conv2D(96, (3, 3), strides=(1, 1), padding='same')(x)
x = LeakyReLU()(x)
x = dropout_and_batch(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
x = LeakyReLU()(x)
x = dropout_and_batch(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Conv2D(96, (3, 3), strides=(1, 1), padding='same')(x)
x = LeakyReLU()(x)
x = dropout_and_batch(x)
x = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(x)
x = LeakyReLU()(x)
x = dropout_and_batch(x)
x = Flatten()(x)

def noiser(args):
    global z_mean, z_log_var
    z_mean, z_log_var = args
    N = K.random_normal(shape=(hidden_dim,), mean=0., stddev=1.0)
    return abs(K.exp(z_log_var / 2) * N + z_mean)

z_mean = Dense(hidden_dim)(x)
z_log_var = Dense(hidden_dim)(x)

h = Lambda(noiser, output_shape=(hidden_dim,))([z_mean, z_log_var])

deco = Input((hidden_dim,))
decod = Dense(25*40)(deco)
decod = LeakyReLU()(decod)
decod = Reshape((25, 40, 1))(decod)
low = Conv2DTranspose(96, (3, 3), strides=(1, 1), padding='same')(decod)
low = LeakyReLU()(low)
low = dropout_and_batch(low)
low = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(low)
low = LeakyReLU()(low)
low = dropout_and_batch(low)
low = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(low)
low = LeakyReLU()(low)
low = dropout_and_batch(low)
high = Conv2DTranspose(32, (5, 5), strides=(4, 4), padding='same')(decod)
high = LeakyReLU()(high)
high = dropout_and_batch(high)
result = Lambda(add, output_shape=(100, 160, 96))([low, high])
result = Conv2DTranspose(96, (3, 3), strides=(1, 1), padding='same')(result)
result = LeakyReLU()(result)
result = dropout_and_batch(result)
d = Conv2DTranspose(3, (3, 3), strides=(1, 1), padding='same')(result)
d = LeakyReLU()(d)
dec = dropout_and_batch(d)

encoder = keras.Model(input_img, h, name='encoder')
decoder = keras.Model(deco, dec, name='decoder')
generator = keras.Model(input_img, decoder(encoder(input_img)), name='generator')

In [ ]:
def generator_loss(fake_output):
    loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return loss

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# обучение
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        real_output = discriminator(images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss


def train(dataset, epochs):
    history = []
    MAX_PRINT_LABEL = 25
    th = BUFFER_SIZE // (BATCH_SIZE * MAX_PRINT_LABEL)

    for epoch in range(1, epochs + 1):
        print(f'{epoch}/{EPOCHS}: ', end='')

        start = time.time()
        n = 0

        gen_loss_epoch = 0
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
            gen_loss_epoch += K.mean(gen_loss)
            if (n % th == 0): print('=', end='')
            n += 1

        history += [gen_loss_epoch / n]
        print(': ' + str(history[-1]))
        print('Время эпохи {} составляет {} секунд'.format(epoch, time.time() - start))

    return history

In [ ]:
# запуск процесса обучения

EPOCHS = 30
history = train(train_dataset, EPOCHS)

plt.plot(history)
plt.grid(True)
plt.show()